## Preprocessing

In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [17]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [18]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [19]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [20]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_vlaues = 500
app_types = application_df['APPLICATION_TYPE'].value_counts()
#Makes of List of apps to repalces after the cutoff values of 7
application_types_to_replace = app_types[app_types < cutoff_vlaues].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

#Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [21]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [22]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [23]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_vlaues = 1000
cls_types = application_df['CLASSIFICATION'].value_counts()
#Makes of List of apps to repalces after the cutoff values of 7
classifications_to_replace = cls_types[cls_types < cutoff_vlaues].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [24]:
#Columns to convert
encode_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']

# Convert categorical data to numeric with `pd.get_dummies`
dummy_columns = pd.get_dummies(application_df[encode_columns])

#Concatenate the original DataFrame with the dummy columns
application_df_encoded = pd.concat([application_df, dummy_columns], axis=1)

In [25]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop(columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'IS_SUCCESSFUL']).values
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [26]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimiation Attempt 1

In [34]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=70, activation="relu", input_dim=43))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 70)                3080      
                                                                 
 dense_10 (Dense)            (None, 25)                1775      
                                                                 
 dense_11 (Dense)            (None, 1)                 26        
                                                                 
Total params: 4881 (19.07 KB)
Trainable params: 4881 (19.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [36]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5714 - accuracy: 0.7185
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7317
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7312
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7331
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5485 - accuracy: 0.7333
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7333
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7340
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accuracy: 0.7346
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5455 - accuracy: 0.7353
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5456 - accura

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5638 - accuracy: 0.7249 - 723ms/epoch - 3ms/step
Loss: 0.5638052821159363, Accuracy: 0.7248979806900024


## Optimiation Attempt 2

In [41]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=90, activation="relu", input_dim=43))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=45, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 90)                3960      
                                                                 
 dense_16 (Dense)            (None, 45)                4095      
                                                                 
 dense_17 (Dense)            (None, 1)                 46        
                                                                 
Total params: 8101 (31.64 KB)
Trainable params: 8101 (31.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [43]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5678 - accuracy: 0.7252
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5540 - accuracy: 0.7312
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5522 - accuracy: 0.7332
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7334
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7338
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7346
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7342
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7349
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7358
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5451 - accura

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5667 - accuracy: 0.7256 - 464ms/epoch - 2ms/step
Loss: 0.5667020678520203, Accuracy: 0.7255976796150208


## Optimiation Attempt 3

In [88]:
# Evaluate the model using the test data
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [89]:
# Evaluate the model using the test data
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [90]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [91]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [92]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_vlaues = 500
app_types = application_df['APPLICATION_TYPE'].value_counts()
#Makes of List of apps to repalces after the cutoff values of 7
application_types_to_replace = app_types[app_types < cutoff_vlaues].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

#Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [93]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [94]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_vlaues = 1000
cls_types = application_df['CLASSIFICATION'].value_counts()
#Makes of List of apps to repalces after the cutoff values of 7
classifications_to_replace = cls_types[cls_types < cutoff_vlaues].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [95]:
# Convert column 'A' to string
application_df['ASK_AMT'] = application_df['ASK_AMT'].astype(str)

In [96]:
# Look at ASK AMT value counts for binning
application_df['ASK_AMT'].value_counts()

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [97]:
# Choose a cutoff value and create a list of ask_amt to be replaced
cutoff_vlaues = 1000
ask_types = application_df['ASK_AMT'].value_counts()
#Makes of List of apps to repalces after the cutoff values
ask_amt_to_replace = ask_types[ask_types < cutoff_vlaues].index.tolist()

# Replace in dataframe
for ask in ask_amt_to_replace:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(ask,"Other")

# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

5000     25398
Other     8901
Name: ASK_AMT, dtype: int64

In [98]:
#Columns to convert
encode_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT']

# Convert categorical data to numeric with `pd.get_dummies`
dummy_columns = pd.get_dummies(application_df[encode_columns])

#Concatenate the original DataFrame with the dummy columns
application_df_encoded = pd.concat([application_df, dummy_columns], axis=1)

In [99]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop(columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL']).values
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [100]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [109]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=180, activation=tf.keras.layers.LeakyReLU(alpha=0.1), input_dim=44))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=90, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))

#adding third hidden layer
nn_model.add(tf.keras.layers.Dense(units=45, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_53 (Dense)            (None, 180)               8100      
                                                                 
 dense_54 (Dense)            (None, 90)                16290     
                                                                 
 dense_55 (Dense)            (None, 45)                4095      
                                                                 
 dense_56 (Dense)            (None, 1)                 46        
                                                                 
Total params: 28531 (111.45 KB)
Trainable params: 28531 (111.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [110]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [111]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5667 - accuracy: 0.7244
Epoch 2/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5542 - accuracy: 0.7303
Epoch 3/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5504 - accuracy: 0.7332
Epoch 4/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5492 - accuracy: 0.7326
Epoch 5/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5490 - accuracy: 0.7322
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7335
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5463 - accuracy: 0.7339
Epoch 8/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5458 - accuracy: 0.7349
Epoch 9/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5458 - accuracy: 0.7355
Epoch 10/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5439 - accura

In [112]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6152 - accuracy: 0.7240 - 770ms/epoch - 3ms/step
Loss: 0.6151952743530273, Accuracy: 0.7239649891853333


In [113]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_Optimizatio.h5")